<a href="https://colab.research.google.com/github/tuntul17/project-cycling/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fitparse
import fitparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fitparse: filename=fitparse-1.2.0-py3-none-any.whl size=68212 sha256=67e7699daa65ddae1eb52bbac488af6821d42ecfae2dc2c0107a92fa71b6f73d
  Stored in directory: /root/.cache/pip/wheels/81/67/7b/77a2f8ba348bafbbad6262a80bc51be27b2f9fccbaefc74671
Successfully built fitparse


In [1]:
import pandas as pd
#import numpy as np
#from pylab import *
#import seaborn as sns
#import folium
import os, shutil
import zipfile
import gzip
#import torch

In [ ]:
folder ="/content/drive/MyDrive/Colab Notebooks/Cycling-adventures/data/raw_data/StravaData"
os.chdir(folder)
df0 = pd.DataFrame()
# Open the ZIP file
for n,f in enumerate(os.listdir()):
  if ".gpx" in f:
    continue
  tablecontent = {}
  with gzip.open(f, 'rb') as gz_file:
    # Read the content of the GZIP file
    F = gz_file.read()
  ff = fitparse.FitFile(F)
  units = []
  for r in ff.get_messages("record"): #r is record
    for d in r:
      # d is data field
      dataname = d.name
      datavalue = d.value
      unit = d.units
      if dataname in tablecontent:
        tablecontent[dataname].append(datavalue)
      else:
        tablecontent[dataname] = [datavalue]
        units.append(unit)
  dum = pd.DataFrame.from_dict(tablecontent, orient='index',).transpose()
  dum["id"] = n
  df0 = pd.concat([df0, dum], ignore_index=True)
df0.to_csv("/content/drive/MyDrive/Colab Notebooks/Cycling-adventures/data/output_data/raw_rides.csv",index=False)
#save the units
U = [{"Feature": feature, "Units": unit}
     for feature, unit in zip(tablecontent.keys(), units)]

# Create a DataFrame from the list of dictionaries
unitdf = pd.DataFrame(U)
unitdf.to_excel("/content/drive/MyDrive/Colab Notebooks/Cycling-adventures/data/output_data/units.xlsx",index=False)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Cycling-adventures/data/output_data/raw_rides.csv") #just to save some time

In [ ]:
#to convert to semicircles to coordinates
#Since all the rides are from North-East semisphere I won't check the signs.
def loc_converter(semi_lat,semi_lon):
  lat = (semi_lat / 2**31) * 180
  lon = (semi_lon/2**31)*180
  return lat, lon

In [ ]:
#do not convert altitude
#do not convert cadence
df["distance"] = df["distance"]/1000 # meters to kilometers
#do not convert heart_rate
new_location = loc_converter(df["position_lat"],df["position_long"])
df["position_lat"] = new_location[0]
df["position_long"] = new_location[1]
df["speed"] = df["speed"]* 3.6 # m/s to km/h
df.drop(["enhanced_altitude","enhanced_speed","power","fractional_cadence","unknown_66","unknown_61","unknown_108"],axis=1,inplace= True)


In [ ]:
missing = pd.DataFrame()
for i in df["id"].unique():
  subset = df[df["id"]==i][df.columns[:-1]]
  miss_rat = (subset.isna().mean()*100).round(3).to_dict()
  dum1 = pd.DataFrame.from_dict(miss_rat,orient='index').transpose()
  dum1["id"] = i
  dum1["time"] = (subset["timestamp"].iloc[0])[:10] #.astype(int)
  missing = pd.concat([dum1,missing],ignore_index=True)
missing = missing.iloc[::-1].reset_index(drop=True)
#Now if any feature missing more than 15% of the each ride, I will those rides as `unusable` and other will be labelled as `usable`.
threshold = 15
missing['Usage'] = missing[missing.columns[:8]].apply(lambda row: 1 if all(row <= threshold) else 0, axis=1)



In [ ]:
usage_mapping = missing.set_index('id')['Usage'].to_dict()
df["Usage"] = df["id"].map(usage_mapping)

df.to_csv("/content/drive/MyDrive/Colab Notebooks/Cycling-adventures/data/output_data/convertedrides.csv",index=False)
missing.to_csv("/content/drive/MyDrive/Colab Notebooks/Cycyling data outputs/missing_values.csv",index=False)

In [ ]:
missing[missing["Usage"]==1]

In [17]:
df0 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Cycling-adventures/data/output_data/convertedrides.csv")

In [6]:
#import the summed data to clean.
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Cycling-adventures/data/raw_data/activities.csv")

In [ ]:
df.drop(["Activity ID",'Activity Name','Commute', 'Activity Private Note', 'Activity Gear','Max Grade', 'Average Grade','Max Temperature', 'Average Temperature',
       'Average Positive Grade', 'Average Negative Grade','Max Heart Rate.1',
       'Filename', 'Athlete Weight', 'Bike Weight', 'Elapsed Time.1','Max Watts', 'Average Watts',"Weighted Average Power", "Power Count", "Prefer Perceived Exertion",
       'Perceived Relative Effort','Activity Description', 'Commute.1', 'Total Weight Lifted',
       'From Upload', 'Grade Adjusted Distance', 'Weather Observation Time',
       'Weather Condition', 'Weather Temperature', 'Apparent Temperature','Distance.1',
       'Dewpoint', 'Humidity', 'Weather Pressure', 'Wind Speed', 'Wind Gust',
       'Wind Bearing', 'Precipitation Intensity', 'Sunrise Time',
       'Sunset Time', 'Moon Phase', 'Bike', 'Gear','Max Grade', 'Average Grade',
       'Average Positive Grade', 'Average Negative Grade','Elevation Low',
       'Elevation High',
       'Precipitation Probability', 'Precipitation Type', 'Cloud Cover',
       'Weather Visibility', 'UV Index', 'Weather Ozone',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.jump_count">Jump Count</span>',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.total_grit">Total Grit</span>',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.avg_flow">Avg Flow</span>',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.flagged">Flagged</span>',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.avg_elapsed_speed">Avg Elapsed Speed</span>',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.dirt_distance">Dirt Distance</span>',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.newly_explored_distance">Newly Explored Distance</span>',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.newly_explored_dirt_distance">Newly Explored Dirt Distance</span>',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.sport_type">Sport Type</span>',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.total_steps">Total Steps</span>',
       'Media','Relative Effort.1', 'Total Work', 'Number of Runs', 'Uphill Time',
       'Downhill Time', 'Other Time', 'Perceived Exertion',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.type">Type</span>',
       '<span class="translation_missing" title="translation missing: en-US.lib.export.portability_exporter.activities.horton_values.start_time">Start Time</span>'],inplace = True,axis = 1)
df

In [ ]:
df[["Activity Date","Activity Type","Distance",'Elevation Gain','Elevation Loss',
              'Elapsed Time','Moving Time','Average Speed','Max Speed','Average Cadence','Max Cadence',
              'Average Heart Rate','Max Heart Rate','Relative Effort','Calories']]
df = df[df["Activity Type"]=='Ride'] #drop the runs
df['Activity Date']=pd.to_datetime(df['Activity Date'], format='%b %d, %Y, %I:%M:%S %p')


In [37]:
merged_df = pd.merge(df, df0[['id', 'Usage','timestamp']], left_on='Activity Date', right_on='timestamp', how='left')

# Drop the redundant timestamp column
df = merged_df.drop(columns=['timestamp'])


In [39]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Cycling-adventures/data/output_data/cleanNsummed.csv",index=False)